In [1]:
import datasets
import transformers

INFO: transformers 4.0.0 on mcsgpu


In [50]:
# dataset = datasets.load_dataset('./data/wikifactcheck', split='train',
#                                 cache_dir='/sw/mcs/wfc/datasets/')

INFO: {'train': '/sw/mcs/wfc/datasets/downloads/737bc15ea4440e012d868c6f61efb6f4b91e4176bc22853a1bb864d9c2448022', 'dev': '/sw/mcs/wfc/datasets/downloads/7a674645a64b0dea3de87f545b162a0d8f60d2396ad75ff3203a76259f84c52f'}



INFO: encountered 20 errors processing /sw/mcs/wfc/datasets/downloads/737bc15ea4440e012d868c6f61efb6f4b91e4176bc22853a1bb864d9c2448022



INFO: encountered 5 errors processing /sw/mcs/wfc/datasets/downloads/7a674645a64b0dea3de87f545b162a0d8f60d2396ad75ff3203a76259f84c52f
Dataset wiki_fact_check downloaded and prepared to /sw/mcs/wfc/datasets/wiki_fact_check/plain_text/1.0.0/7158b728cbd5e133282afd03da45069799a4cfaed9966aef4df51ee156d0d2fa. Subsequent calls will reuse this data.


In [58]:
dataset

Dataset({
    features: ['id', 'claim', 'context', 'evidence', 'label'],
    num_rows: 38916
})

In [54]:
dataset.save_to_disk('/sw/mcs/wfc/datasets/wfc_train_with_evidence')

In [56]:
# dataset_dev = datasets.load_dataset('./data/wikifactcheck', split='validation',
#                                     cache_dir='/sw/mcs/wfc/datasets/')

Reusing dataset wiki_fact_check (/sw/mcs/wfc/datasets/wiki_fact_check/plain_text/1.0.0/7158b728cbd5e133282afd03da45069799a4cfaed9966aef4df51ee156d0d2fa)


In [57]:
dataset_dev

Dataset({
    features: ['id', 'claim', 'context', 'evidence', 'label'],
    num_rows: 9730
})

In [59]:
dataset_dev.save_to_disk('/sw/mcs/wfc/datasets/wfc_dev_with_evidence')

### now try to train a BERT?

In [2]:
import torch
import datasets
from transformers import (
    AutoTokenizer,
    AutoModel,
    AutoModelForSequenceClassification
)

INFO: transformers 4.0.0 on mcsgpu


In [3]:
dataset = datasets.load_from_disk('/sw/mcs/wfc/datasets/wfc_train_with_evidence')
dataset_dev = datasets.load_from_disk('/sw/mcs/wfc/datasets/wfc_dev_with_evidence')

In [4]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/distilbert-base-nli-stsb-mean-tokens', 
                                          max_length=128)

def tokenize_wfc(e, tokenizer=tokenizer):
    cl = tokenizer(e['claim'], truncation=True, padding='max_length')
    ev = map(lambda sent: tokenizer(sent, truncation=True, padding='max_length'),
             e['evidence'].split('. '))
    return {
        **cl,
        'evidence': list(ev)
    }

In [1]:
dataset_tok = dataset.map(tokenize_wfc, batched=False)
dataset_dev_tok = dataset_dev.map(tokenize_wfc, batched=False)

# dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

NameError: name 'dataset' is not defined

In [ ]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=8)

In [3]:
model = AutoModel.from_pretrained('sentence-transformers/distilbert-base-nli-stsb-mean-tokens')

In [4]:
model

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0): TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(i

In [18]:
model.forward

Signature:
model.forward(
    input_ids=None,
    attention_mask=None,
    head_mask=None,
    inputs_embeds=None,
    output_attentions=None,
    output_hidden_states=None,
    return_dict=None,
)
Docstring:
The :class:`~transformers.DistilBertModel` forward method, overrides the :func:`__call__` special method.

.. note::
    Although the recipe for forward pass needs to be defined within this function, one should call the
    :class:`Module` instance afterwards instead of this since the former takes care of running the pre and post
    processing steps while the latter silently ignores them.
    
Args:
    input_ids (:obj:`torch.LongTensor` of shape :obj:`(batch_size, num_choices)`):
        Indices of input sequence tokens in the vocabulary.

        Indices can be obtained using :class:`~transformers.DistilBertTokenizer`. See
        :meth:`transformers.PreTrainedTokenizer.encode` and :meth:`transformers.PreTrainedTokenizer.__call__` for
        details.

        `What are input I

## language model

In [5]:
import torch
import datasets
from transformers import (
    AutoTokenizer,
    AutoModelWithLMHead,
    AutoModelForSequenceClassification
)

In [12]:
tok = AutoTokenizer.from_pretrained('./wfclm/bert_5000_lr=1e-05/checkpoint-150000', max_length=128)

In [7]:
lm = AutoModelWithLMHead.from_pretrained('./wfclm/bert_5000_lr=1e-05/checkpoint-150000')

/home/as9kc/code/wfc-hons/transformers/models/auto/modeling_auto.py:852: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [58]:
enc = tok.batch_encode_plus([f'The hindwings are flat with a {tok.mask_token} line across the dorsal'], 
                            padding='max_length', truncation=True, max_length=128)
enc = {k: torch.Tensor(v).long() for k,v in enc.items()}

In [59]:
lmout = lm(**enc)

In [60]:
tok.decode(torch.argmax(lmout.logits, dim=2).view(-1))

'. the hindwings are grey with a dark line of the dorsal.................... a.. a a... a..................... a.................................................................'

# Fresh Start to Pipeline

In [74]:
import numpy as np
import torch
from sklearn.metrics import f1_score, confusion_matrix, precision_score, recall_score

In [2]:
from datasets import load_dataset, load_from_disk
from data.wfc_processor import wfc_processors as processors
from data.wfc_processor import wfc_output_modes as output_modes
from data.wfc_processor import wfc_convert_examples_to_features

INFO: transformers 4.0.0 on mcsgpu


In [3]:
from sentence_transformers import SentenceTransformer, util
sbert = SentenceTransformer('paraphrase-distilroberta-base-v1').cuda()

In [106]:
from transformers import pipeline
nli = pipeline("zero-shot-classification")
qa = pipeline("question-answering")

from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
config = AutoConfig.from_pretrained('roberta-large-mnli', max_length=512)
tokenizer = AutoTokenizer.from_pretrained('roberta-large-mnli')
model = AutoModelForSequenceClassification.from_pretrained('roberta-large-mnli', config=config).cuda()

Some weights of the model checkpoint at facebook/bart-large-mnli were not used when initializing BartModel: ['model.encoder.version', 'model.decoder.version']
- This IS expected if you are initializing BartModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BartModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at facebook/bart-large-mnli were not used when initializing BartForSequenceClassification: ['model.encoder.version', 'model.decoder.version']
- This IS expected if you are initializing BartForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification m

In [107]:
proc = processors['wfc']()

In [108]:
train = proc.get_train_examples('data')
valid = proc.get_dev_examples('data')

0it [00:00, ?it/s]

INFO: get_examples train both=True


100it [00:00, 221.26it/s]
22it [00:00, 205.32it/s]

INFO: get_examples dev both=True


100it [00:00, 242.97it/s]


In [109]:
k = 3
Y_hat = []
Y = []

sbert.eval()
model.eval()

for i, (example) in enumerate(valid):
    print(f'Classifying example {i}/{len(valid)} \r', end='')
    cos_scores = []
    text_a = example.text_a
    a_emb = sbert.encode(example.text_a, device='cuda')
    b_emb = sbert.encode(example.text_b, device='cuda')
    
    cos = util.pytorch_cos_sim(a_emb, b_emb).view(-1)  
    
#     print(f'obtained {len(cos)} cosine similarity scores for length {len(example.text_b)} evidence \r', end='')
    
    top_cos = np.argsort(-cos)[:k]
    
    for ix in top_cos:
#         print(f'Performing classification on index {ix} \r', end='')
        text_b = example.text_b[ix]
        
        inputs = tokenizer.encode_plus(text=text_b, text_pair=text_a, padding='max_length', return_tensors='pt')
        inputs = {k: v.cuda() for k,v in inputs.items()}
        
        y = model(**inputs)
#         print(text_a, text_b, '\n', y)
        Y += [np.array(y.logits.detach().cpu().view(-1))]
    Y_hat += [example.label]

In [110]:
tokenizer.decode(tokenizer.encode_plus(text=text_a, text_pair=text_b, padding='max_length', return_tensors='pt')['input_ids'][0])

'<s>The risk is eliminated with congenital polyposis syndromes, and is increased in acquired causes.</s></s> High-grade dysplasia associated with fundic gland polyposis in a familial adenomatous polyposis patient, with special reference to APC mutation profiles</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

#### Now we must convert each logit to a label

In [111]:
print(config.id2label)
Y_ = torch.Tensor([Y[i:i+3] for i in range(0, len(Y), 3)])
labels = torch.argmax(Y_, axis=2)
binarized = torch.any(labels == 2, axis=1).double()
Y_hat_dbl = torch.Tensor([item == 'supported' for item in Y_hat])

{0: 'CONTRADICTION', 1: 'NEUTRAL', 2: 'ENTAILMENT'}


In [122]:
def accuracy(A,B): return sum(A == B) / len(A)
for metric in [accuracy, f1_score, precision_score, recall_score]:
    print(f'{str(metric):<45} {metric(binarized, Y_hat_dbl)*100:.2f}')


<function accuracy at 0x7f001654f700>         60.61
<function f1_score at 0x7f001ffadd30>         38.10
<function precision_score at 0x7f001ffb4280>  24.24
<function recall_score at 0x7f001ffb43a0>     88.89


In [113]:
confusion_matrix(binarized, Y_hat_dbl)

array([[96, 75],
       [ 3, 24]])

### Now we'll use some scipy metrics to evaluate this output

In [25]:
len(Y), len(Y_hat)

(594, 198)